In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
## Importing Libraries
import shutil
import cv2
import zipfile
import numpy as np
import os

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPool2D, UpSampling2D, Concatenate, Add


In [3]:
shutil.copyfile('/content/drive/MyDrive/Camouflage_Detection/kaggle.json','/content/kaggle.json')

'/content/kaggle.json'

In [4]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
! kaggle datasets download -d boatshuai/cod10k-v3

Dataset URL: https://www.kaggle.com/datasets/boatshuai/cod10k-v3
License(s): apache-2.0
100% 2.25G/2.26G [00:29<00:00, 138MB/s]
100% 2.26G/2.26G [00:29<00:00, 81.5MB/s]


In [5]:
with zipfile.ZipFile('/content/cod10k-v3.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')

In [6]:
## Defining Functions

## To Read an image
def apply_visual_attention(path):
    img = cv2.imread(path, 0)
    return img

## To convert an image from BGR to Gray
def feature(image):
    # image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return image


## To load a limited number of Image and Mask from Directory
def load_data(image_path, gt_path,maxi):
    images = []
    gt_instances = []
    c=0
    for filename in sorted(os.listdir(image_path)):
        c+=1
        if(c>=maxi):
            break
        if filename.endswith('.jpg'):
            # Load image and resize to (256,256)
            img=apply_visual_attention(os.path.join(image_path , filename))
            img = cv2.resize(img, (256,256))
            images.append(img)
            # Load ground truth instance and resize to (256,256)
            gt = cv2.imread(os.path.join(gt_path ,filename[:-4] + '.png'), cv2.IMREAD_GRAYSCALE)
            gt = cv2.resize(gt, (256,256), interpolation=cv2.INTER_LINEAR)
            gt_instances.append(gt)
    return np.array(images), np.array(gt_instances)


In [7]:

def Conv2D_block(Input,Output_shape,rate=1):
  X=Conv2D(Output_shape,3,padding='same',dilation_rate=rate)(Input)
  ###______dilation_rate=rate_given_as _parameter
  X=BatchNormalization()(X)
  X=Activation('relu')(X)
  return X

def Downsample_Conv2D_block(Input,Output_shape,rate=1):
  X=MaxPool2D((2,2))(Input)
  X=Conv2D(Output_shape,3,padding='same',dilation_rate=rate)(X)
  X=BatchNormalization()(X)
  X=Activation('relu')(X)
  return X


def Upsample_Conv2D_block(Input):
  X=UpSampling2D(size=(2, 2), interpolation="bilinear")(Input)
  return X


def RSU_L(Input,Input_shape,Output_shape,num_of_layers,rate=2):
  ## Initial Block
  X=Conv2D_block(Input,Output_shape)
  save_1=X


  ## Layers Block
  ## Going Down Through Encoder
  Saving_output=[]

  X=Conv2D_block(X,Input_shape)
  Saving_output.append(X)

  for i in range(num_of_layers-2):
    X=Downsample_Conv2D_block(X,Input_shape)
    Saving_output.append(X)

  ## Last Block
  X=Conv2D_block(X,Input_shape,rate=rate)
  ##Moving Up Through Encoder

  X=Concatenate()([X,Saving_output.pop()])
  X=Conv2D_block(X,Input_shape)


  for i in range(num_of_layers-3):
    X=Upsample_Conv2D_block(X)
    X=Concatenate()([X,Saving_output.pop()])
    X=Conv2D_block(X,Input_shape)

  X=Upsample_Conv2D_block(X)
  X=Concatenate()([X,Saving_output.pop()])
  X=Conv2D_block(X,Output_shape)

  ## Encoder Close

  X=Add()([X,save_1])

  return X



def RSU_4F(Input,Input_shape,Output_shape):
  Saving_output=[]

  X=Conv2D_block(Input,Output_shape)
  Saving_output.append(X)
  X=Conv2D_block(X,Input_shape)
  Saving_output.append(X)

  X=Conv2D_block(X,Input_shape,2)
  Saving_output.append(X)
  X=Conv2D_block(X,Input_shape,4)
  Saving_output.append(X)


  X=Conv2D_block(X,Input_shape,8)


  X=Concatenate()([X,Saving_output.pop()])
  X=Conv2D_block(X,Input_shape,4)

  X=Concatenate()([X,Saving_output.pop()])
  X=Conv2D_block(X,Input_shape,2)

  X=Concatenate()([X,Saving_output.pop()])
  X=Conv2D_block(X,Output_shape)

  X=Add()([X,Saving_output.pop()])

  return X

In [8]:
def U_sq_net(Input_dim,Input_shape,Output_shape,num_classes=1):
  ## Input Layer
  Inputs=Input(Input_dim)
  s0=Inputs

  ## Encoder
  S1=RSU_L(s0,Input_shape[0],Output_shape[0],7)
  P1=MaxPool2D((2,2))(S1)

  S2=RSU_L(P1,Input_shape[1],Output_shape[1],6)
  P2=MaxPool2D((2,2))(S2)

  S3=RSU_L(P2,Input_shape[2],Output_shape[2],5)
  P3=MaxPool2D((2,2))(S3)

  S4=RSU_L(P3,Input_shape[3],Output_shape[3],4)
  P4=MaxPool2D((2,2))(S4)

  S5=RSU_4F(P4,Input_shape[4],Output_shape[4])
  P5=MaxPool2D((2,2))(S5)

  ## Brdige
  B1=RSU_4F(P5,Input_shape[5],Output_shape[5])
  B2 = Upsample_Conv2D_block(B1)

  ## Decoder
  D1=Concatenate()([B2,S5])
  D1=RSU_4F(D1,Input_shape[6],Output_shape[6])
  U1=Upsample_Conv2D_block(D1)

  D2=Concatenate()([U1,S4])
  D2=RSU_L(D2,Input_shape[7],Output_shape[7],4)
  U2=Upsample_Conv2D_block(D2)

  D3=Concatenate()([U2,S3])
  D3=RSU_L(D3,Input_shape[8],Output_shape[8],5)
  U3=Upsample_Conv2D_block(D3)

  D4=Concatenate()([U3,S2])
  D4=RSU_L(D4,Input_shape[9],Output_shape[9],6)
  U4=Upsample_Conv2D_block(D4)

  D5=Concatenate()([U4,S1])
  D5=RSU_L(D5,Input_shape[10],Output_shape[10],7)

  ## Sidewise Output
  y1 = Conv2D(num_classes, 3, padding="same")(D5)
  y1 = Activation("sigmoid")(y1)

  y2 = Conv2D(num_classes, 3, padding="same")(D4)
  y2 = UpSampling2D(size=(2, 2), interpolation="bilinear")(y2)
  y2 = Activation("sigmoid")(y2)

  y3 = Conv2D(num_classes, 3, padding="same")(D3)
  y3 = UpSampling2D(size=(4, 4), interpolation="bilinear")(y3)
  y3 = Activation("sigmoid")(y3)

  y4 = Conv2D(num_classes, 3, padding="same")(D2)
  y4 = UpSampling2D(size=(8, 8), interpolation="bilinear")(y4)
  y4 = Activation("sigmoid")(y4)

  y5 = Conv2D(num_classes, 3, padding="same")(D1)
  y5 = UpSampling2D(size=(16, 16), interpolation="bilinear")(y5)
  y5 = Activation("sigmoid")(y5)

  y6 = Conv2D(num_classes, 3, padding="same")(B1)
  y6 = UpSampling2D(size=(32, 32), interpolation="bilinear")(y6)
  y6 = Activation("sigmoid")(y6)

  y0 = Concatenate()([y1, y2, y3, y4, y5, y6])
  y0 = Conv2D(num_classes, 3, padding="same")(y0)
  y0 = Activation("sigmoid")(y0)


  model = tf.keras.models.Model(Inputs, outputs=[y0, y1, y2, y3, y4, y5, y6])
  return model


In [9]:
def Build_U_sq_net(Input_dim, num_classes=1):
    out_ch = [64, 128, 256, 512, 512, 512, 512, 256, 128, 64, 64]
    int_ch = [32, 32, 64, 128, 256, 256, 256, 128, 64, 32, 16]
    model = U_sq_net(Input_dim, int_ch,out_ch, num_classes=num_classes)
    return model

In [10]:
model=Build_U_sq_net((256,256,1))
model.summary()
model.save('U_sq_net_model.keras')

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 256, 256, 1)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 256, 256, 64)   │            640 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 256, 256, 64)   │            256 │ conv2d[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation (Activation)   │ (None, 256, 256, 64)   │              0 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 256, 256, 32)   │         18,464 │ activation[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 256, 256, 32)   │            128 │ conv2d_1[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_1 (Activation) │ (None, 256, 256, 32)   │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 128, 128, 32)   │              0 │ activation_1[0][0]     │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 128, 128, 32)   │          9,248 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 128, 128, 32)   │            128 │ conv2d_2[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_2 (Activation) │ (None, 128, 128, 32)   │              0 │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_1           │ (None, 64, 64, 32)     │              0 │ activation_2[0][0]     │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 64, 64, 32)     │          9,248 │ max_pooling2d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_3     │ (None, 64, 64, 32)     │            128 │ conv2d_3[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_3 (Activation) │ (None, 64, 64, 32)     │              0 │ batch_normalization_3… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_2           │ (None, 32, 32, 32)     │              0 │ activation_3[0][0]     │
│ (MaxPooling2D)            │                        │                │                        │
├──────────────────────

 Total params: 44,037,565 (167.99 MB)

 Trainable params: 44,008,765 (167.88 MB)

 Non-trainable params: 28,800 (112.50 KB)

In [11]:
train_image_path = '/content/COD10K-v3/Train/Image'
train_gt_path = '/content/COD10K-v3/Train/GT_Object'
test_image_path = '/content/COD10K-v3/Test/Image'
test_gt_path = '/content/COD10K-v3/Test/GT_Object'

In [12]:
train_images, train_gt_instances = load_data(train_image_path, train_gt_path,6000)
# test_images, test_gt_instances = load_data(test_image_path, test_gt_path,1000)

In [13]:
train_images = np.stack(train_images, axis=0)
train_masks = np.stack(train_gt_instances, axis=0)

In [ ]:

train_images = train_images.astype('float32') / 255.0
train_masks = train_masks.astype('float32') / 255.0
train_masks = np.expand_dims(train_masks, axis=-1)

model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(train_images, [train_masks,train_masks,train_masks,train_masks,train_masks,train_masks,train_masks], epochs=10, batch_size=4, validation_split=0.2) # Adjust epochs and batch size as needed

model.save('trained_u_sq_net_model.keras')

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy']*7)
model.fit(train_images, train_masks, epochs=10, validation_split=0.2)

Epoch 1/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 835s 4s/step - activation_118_accuracy: 0.1084 - loss: -18.5381 - val_activation_118_accuracy: 0.0000e+00 - val_loss: 4.4798
Epoch 2/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 363s 2s/step - activation_118_accuracy: 1.5120e-05 - loss: -76.1041 - val_activation_118_accuracy: 0.0000e+00 - val_loss: 8.4777
Epoch 3/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 381s 2s/step - activation_118_accuracy: 1.5432e-05 - loss: -127.0990 - val_activation_118_accuracy: 0.0000e+00 - val_loss: 12.5448
Epoch 4/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 397s 2s/step - activation_118_accuracy: 1.5784e-05 - loss: -186.0203 - val_activation_118_accuracy: 0.0000e+00 - val_loss: 15.8677
Epoch 5/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 382s 2s/step - activation_118_accuracy: 1.4782e-05 - loss: -210.8253 - val_activation_118_accuracy: 0.0000e+00 - val_loss: 15.8705
Epoch 6/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 367s 2s/step - activation_118_accuracy: 1.4094e-05 - loss: -206.9884 - val_activation_118_accuracy: 0.0000e+00 - 

In [ ]:
model.save('Weighted_U_sq_net_model.keras')
shutil.copyfile('/content/Weighted_U_sq_net_model.keras','/content/drive/MyDrive/Camouflage_Detection/Weighted_U_sq_net_model.keras')

In [ ]:
from tf.keras.models import load_model
model = load_model('/content/drive/MyDrive/Camouflage_Detection/Weighted_U_sq_net_model.keras')

In [ ]:
def find_result(img_path,generator):
  otimg=apply_visual_attention(img_path)
  timg = cv2.resize(otimg, (256,256))
  timg=np.array([timg])
  predictions = generator.predict(timg)
  return predictions[0]

In [ ]:

image_path = '/content/COD10K-v3/Train/Image/COD10K-CAM-1-Aquatic-1-BatFish-3.jpg'
find_result_output = find_result(image_path, model)

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
original_image = cv2.imread(image_path)
original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
plt.imshow(cv2.resize(original_image, (512,512)))
plt.title('Original Image')

plt.subplot(1, 2, 2)
plt.imshow(find_result_output, cmap='gray')
plt.title('find_result Output')

plt.show()